In [ ]:
# get the dataset
!gdown --id 1xRoxdM92JeDqKjEIguB4ubKuBb2gF7os
!unzip dataset.zip

In [ ]:
import pandas as pd # importing
import matplotlib.pyplot as plt

# **Loading Dataset**

In [ ]:
df_installs = pd.read_csv('/content/dataset/installs.txt', sep='\t')
df_app_starts = pd.read_csv('/content/dataset/app starts.txt', sep='\t')
df_brochure_views = pd.read_csv('/content/dataset/brochure views.txt', sep='\t')
df_app_starts_july = pd.read_csv('/content/dataset/app starts july.txt', sep='\t')
df_brochure_views_july = pd.read_csv('/content/dataset/brochure views july.txt', sep='\t')

Fixing datetime type

In [ ]:
df_installs['InstallDate'] = pd.to_datetime(df_installs['InstallDate'])
df_app_starts['dateCreated'] = pd.to_datetime(df_app_starts['dateCreated'])
df_brochure_views['dateCreated'] = pd.to_datetime(df_brochure_views['dateCreated'])
df_brochure_views_july['dateCreated'] = pd.to_datetime(df_brochure_views_july['dateCreated'])
df_app_starts_july['dateCreated'] = pd.to_datetime(df_app_starts_july['dateCreated'])

In [ ]:
df_installs.head()

fixing null values

In [ ]:
# there are some null values in the borchure view_duration column
df_brochure_views.fillna(0,inplace=True)
df_brochure_views_july.fillna(0,inplace=True)

### **a little EDA**

In [ ]:
# Extract day and hour
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_app_starts['day_of_week'] = df_app_starts['dateCreated'].dt.day_name()
df_app_starts['day_of_week'] = pd.Categorical(df_app_starts['day_of_week'], categories=ordered_days, ordered=True)
df_app_starts['hour'] = df_app_starts['dateCreated'].dt.hour

# Plot day of week
df_app_starts['day_of_week'].value_counts().sort_index().plot(kind='bar')

plt.title('Popular Days')
plt.xlabel('Day of the Week')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Plot hour of the day
df_app_starts['hour'].value_counts().sort_index().plot(kind='bar')
plt.title('Popular Hours')
plt.show()

In [ ]:
# Extract day and hour
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_brochure_views['day_of_week'] = df_brochure_views['dateCreated'].dt.day_name()
df_brochure_views['day_of_week'] = pd.Categorical(df_brochure_views['day_of_week'], categories=ordered_days, ordered=True)
df_brochure_views['hour'] = df_brochure_views['dateCreated'].dt.hour

# Plot day of week
df_brochure_views['day_of_week'].value_counts().sort_index().plot(kind='bar')

plt.title('Popular Days')
plt.xlabel('Day of the Week')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Plot hour of the day
df_brochure_views['hour'].value_counts().sort_index().plot(kind='bar')
plt.title('Popular Hours')
plt.show()

In [ ]:
# Plot hour of the day
df_brochure_views['view_duration'].plot(kind='bar')
plt.title('Popular Hours')
plt.show()

# **Feature Creation**

To calculate churn, what are useful features?
* Customer_Lifetime (days since app installed/joined)
* app_open_count, weekly_app_opening_rate (Frequency), days_since_last_opened (Recency), install_to_open_gap
* brochur_view_count, weekly_brochur_view_rate ,commulative_broucher_view_duration, avg_broucher_view_time, days_since_last_brochur_viewed, maximum_visited_broshur



---


**User-Base** : list of all the user's we are considering for the problem.


---




In [ ]:
# User-Base (userId, productId, model, campaignId)
user_base_df = df_installs[['userId', 'productId', 'model', 'campaignId','InstallDate']].copy()



---



**days_since_app_installed**: How many days have been passed since user joined app.

---




In [ ]:
max_dataset_date = df_app_starts['dateCreated'].max() # getting dataset's last date

In [ ]:
# Customer Lifetime (total days since user joined app)
user_base_df['days_since_app_installed'] = (max_dataset_date - user_base_df['InstallDate']).dt.days



---

**lifetime_app_start_count: How many times has a user started the app in total**

---




In [ ]:
# calculating total app start count for each user
user_base_df_temp = user_base_df.copy()
new_feature_column_name = 'lifetime_app_start_count' # to keep it centralized
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

app_start_count_df = df_app_starts.groupby('userId', as_index=False).agg({'dateCreated': 'count'})
app_start_count_df.rename(columns={'userId': 'userId', 'dateCreated': new_feature_column_name}, inplace=True)
# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, app_start_count_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''



---


**avg_daily_app_start** : what's the average of user to open the app on daily basis in his app lifetime (days)

In [ ]:
user_base_df['avg_daily_app_start'] =  user_base_df['lifetime_app_start_count'] / user_base_df['days_since_app_installed']



---


**days_since_last_start (Recency) : How many days have passed since the user opened the application last time**


---




In [ ]:
new_feature_column_name = 'days_since_last_start'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

# aggregate the data
last_app_start_date_df = df_app_starts.groupby('userId',as_index=False).agg({'dateCreated': 'max'})
last_app_start_date_df.rename(columns={'userId': 'userId', 'dateCreated': 'last_app_start_date'}, inplace=True)
last_app_start_date_df[new_feature_column_name] = (max_dataset_date - last_app_start_date_df['last_app_start_date']).dt.days

# merge into base set and fill missing values
user_base_df_temp = pd.merge(user_base_df_temp,last_app_start_date_df,on='userId',how='left')
user_base_df_temp[new_feature_column_name].fillna(user_base_df_temp['days_since_app_installed'], inplace=True) # fill 0 to user's who dont exist in app_start dataset.
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''



---


**avg_days_between_app_starts (Recency)** : How many days have passed since the user opened the application last time


---




In [ ]:
#avg_time_between_app_starts
new_feature_column_name = 'avg_days_between_app_starts'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

# aggregate the data
df_app_starts_temp = df_app_starts.copy()
df_app_starts_temp = df_app_starts_temp.sort_values(['userId', 'dateCreated'])
df_app_starts_temp[new_feature_column_name] = df_app_starts_temp.groupby('userId')['dateCreated'].diff()
avg_time_diff = df_app_starts_temp.groupby('userId')[new_feature_column_name].mean().dt.total_seconds() / (24 * 3600)
avg_time_diff = avg_time_diff.reset_index()

# merge into base set and fill missing values
user_base_df_temp = pd.merge(user_base_df_temp,avg_time_diff,on='userId',how='left')
user_base_df_temp[new_feature_column_name].fillna(user_base_df_temp['days_since_app_installed'], inplace=True) # fill 0 to user's who dont exist in app_start dataset.
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''

In [ ]:
user_base_df.sort_values('avg_days_between_app_starts')



---



*   **first_app_start_date** : date at which the user first time started app
*   **install_to_open_gap** : days between installing and first time opened the app






---




In [ ]:
new_feature_column_name_1 = 'first_app_start_date'
new_feature_column_name_2 = 'install_to_open_gap'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name_1, axis=1, inplace=True, errors='ignore')
user_base_df_temp.drop(new_feature_column_name_2, axis=1, inplace=True, errors='ignore')

# calculate first time app started
last_app_start_date_df = df_app_starts.groupby('userId',as_index=False).agg({'dateCreated': 'min'})
last_app_start_date_df.rename(columns={'userId': 'userId', 'dateCreated': new_feature_column_name_1}, inplace=True)
user_base_df_temp = pd.merge(user_base_df_temp,last_app_start_date_df,on='userId',how='left')
# calculating install to first start gap (days)
user_base_df_temp[new_feature_column_name_2] = (user_base_df_temp[new_feature_column_name_1].dt.date - user_base_df_temp['InstallDate'].dt.date).dt.days # there was a weird isseu related to getting -1 from the difference (when not used .dt.date)

# there are some users who might have resinstalled app, leaving some entries in app_start.txt.. This resulted in some values being negative in install_to_open_gap column, so fixing
user_base_df_temp.loc[user_base_df_temp[new_feature_column_name_2] < 0, new_feature_column_name_2] = 0
user_base_df_temp[new_feature_column_name_2].fillna(user_base_df_temp['days_since_app_installed'], inplace=True) # fill customer_lifetime_days to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name_1] = user_base_df_temp[new_feature_column_name_1]
user_base_df[new_feature_column_name_2] = user_base_df_temp[new_feature_column_name_2]
user_base_df.sort_values(new_feature_column_name_2)
new_feature_column_name_1 = ''
new_feature_column_name_2 = ''



---

** Now all feature for the Brochures **:

*   lifetime_brochur_view_count
*   daily_brochur_view_rate
*   commulative_broucher_view_duration
*   avg_broucher_view_time
*   days_since_last_brochur_viewed
*   maximum_visited_broshur



---





---


**lifetime_brochur_view_count** : total brochures visited since using the app


---



In [ ]:
# lifetime_brochur_view_count
new_feature_column_name = 'lifetime_brochur_view_count'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

total_brochur_count_df = df_brochure_views.groupby('userId',as_index=False).agg({'id':'count'})
total_brochur_count_df.rename(columns={'userId':'userId','id':new_feature_column_name},inplace= True)
total_brochur_count_df
# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, total_brochur_count_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
# user_base_df.sort_values('lifetime_brochur_view_count')
new_feature_column_name = ''



---


**avg_daily_brochure_view** : on average how many brochures did user visit since install


---



In [ ]:
# avg_daily_brochure_view
user_base_df['avg_daily_brochure_view'] =  user_base_df['lifetime_brochur_view_count'] / user_base_df['days_since_app_installed']



---


**commulative_broucher_view_duration_seconds** : total time (seconds) spent on viewing brochures.


---



In [ ]:
# commulative_broucher_view_duration_seconds
new_feature_column_name = 'commulative_broucher_view_duration_seconds'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

# aggregating data
total_brochur_count_df = df_brochure_views.groupby('userId',as_index=False).agg({'view_duration':'sum'})
total_brochur_count_df.rename(columns={'userId':'userId','view_duration':new_feature_column_name},inplace= True)

# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, total_brochur_count_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name] / 60 # converting origical milisecond values to seconds
new_feature_column_name = ''
# user_base_df.sort_values(new_feature_column_name)



---


**avg_broucher_view_duration** : on average how much time did the user spend on each borchure view


---



In [ ]:
new_feature_column_name = 'avg_broucher_view_duration_seconds'
user_base_df[new_feature_column_name] = user_base_df['commulative_broucher_view_duration_seconds'] / user_base_df['lifetime_brochur_view_count']
user_base_df[new_feature_column_name].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
new_feature_column_name = ''
# user_base_df.iloc[:,-5:].sort_values(new_feature_column_name)[700:710]



---


**days_since_last_brochur_viewed** : how many days have passed since last brochure visit


---



In [ ]:
# days_since_last_brochur_viewed
new_feature_column_name = 'days_since_last_brochur_viewed'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

last_app_start_date_df = df_brochure_views.groupby('userId',as_index=False).agg({'dateCreated': 'max'})
last_app_start_date_df.rename(columns={'userId': 'userId', 'dateCreated': 'last_brochure_view_date'}, inplace=True)
last_app_start_date_df[new_feature_column_name] = (max_dataset_date.date() - last_app_start_date_df['last_brochure_view_date'].dt.date).dt.days # was having values less than 0 for some silly reason, needed to add .dt.date which fixed issue
user_base_df_temp = pd.merge(user_base_df_temp,last_app_start_date_df,on='userId',how='left')
user_base_df_temp[new_feature_column_name].fillna(user_base_df_temp['days_since_app_installed'], inplace=True) # fill 0 to user's who dont exist in app_start dataset.
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''
# user_base_df.sort_values(new_feature_column_name)



---


**maximum_visited_brochure** : which brochure was the user most interested in and have visited it more than others


---



In [ ]:
# # maximum_visited_brochure and visit count
new_feature_column_name_1 = 'max_visited_brochure_id'
new_feature_column_name_2 = 'max_visited_brochure_visit_count'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name_1, axis=1, inplace=True, errors='ignore')
user_base_df_temp.drop(new_feature_column_name_2, axis=1, inplace=True, errors='ignore')

agg_df = df_brochure_views.groupby(['userId','brochure_id']).size().reset_index(name='this_brochur_view_count')
agg_df = agg_df.sort_values(['userId', 'this_brochur_view_count'], ascending=[True, False])
agg_df = agg_df.drop_duplicates(subset='userId', keep='first')
agg_df.rename(columns={'userId':'userId','brochure_id':new_feature_column_name_1,'this_brochur_view_count':new_feature_column_name_2},inplace= True)

# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, agg_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name_2].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).

# setting new feature
user_base_df[new_feature_column_name_1] = user_base_df_temp[new_feature_column_name_1]
user_base_df[new_feature_column_name_2] = user_base_df_temp[new_feature_column_name_2]
new_feature_column_name_1 = ''
new_feature_column_name_2 = ''
# user_base_df.sort_values(new_feature_column_name_2)



---




*   **brochure_id_with_max_duration** : which brochure did the user spent (commulative) maximum duration on
*   **maximum_duration_for_brochure_id** : what was the commulative time spend on the maximum duration brochure





---

In [ ]:
# max_duration_brochure_commulative_duration_seconds and brochure_id_with_max_duration
new_feature_column_name_1 = 'max_duration_brochure_id'
new_feature_column_name_2 = 'max_duration_brochure_commulative_duration_seconds'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name_1, axis=1, inplace=True, errors='ignore')
user_base_df_temp.drop(new_feature_column_name_2, axis=1, inplace=True, errors='ignore')

# aggregating values
agg_df = df_brochure_views.groupby(['userId','brochure_id']).agg(total_view_duration_this_brochure=('view_duration', 'sum')).reset_index()
agg_df = agg_df.sort_values(['userId', 'total_view_duration_this_brochure'], ascending=[True, False])
agg_df = agg_df.drop_duplicates(subset='userId', keep='first')
agg_df.rename(columns={'userId':'userId','brochure_id':new_feature_column_name_1,'total_view_duration_this_brochure':new_feature_column_name_2},inplace= True)

# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, agg_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name_2].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).

# setting new feature
user_base_df[new_feature_column_name_1] = user_base_df_temp[new_feature_column_name_1]
user_base_df[new_feature_column_name_2] = user_base_df_temp[new_feature_column_name_2] / 60
new_feature_column_name_1 = ''
new_feature_column_name_2 = ''
# user_base_df.sort_values(new_feature_column_name_2)



---


**brochure_view_rate_per_app_start** : how many brochur on average does the person sees when he opens the app


---

In [ ]:
# brochure_view_rate_per_app_start
new_feature_column_name = 'brochure_view_rate_per_app_start'
user_base_df[new_feature_column_name] =  user_base_df['lifetime_brochur_view_count'] / user_base_df['lifetime_app_start_count']
user_base_df[new_feature_column_name].fillna(0,inplace=True)
new_feature_column_name = ''



---


**total_page_turn_count** : total pages turned in whole customer life


---

In [ ]:
# total_page_turn_count??
# commulative_broucher_view_duration_seconds
new_feature_column_name = 'total_page_turn_count'
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

# aggregating data
agg_df = df_brochure_views.groupby('userId',as_index=False).agg({'page_turn_count':'sum'})
agg_df.rename(columns={'userId':'userId','page_turn_count':new_feature_column_name},inplace= True)

# merging to user_base
user_base_df_temp = pd.merge(user_base_df_temp, agg_df, on='userId', how='left')
user_base_df_temp[new_feature_column_name].fillna(0, inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''
# user_base_df[user_base_df['total_page_turn_count'] > 0].sort_values(new_feature_column_name)



---


**Average_total_page_turn_per_brochur_view**


---

In [ ]:
new_feature_column_name = 'average_page_turn_per_brochur_view'
user_base_df[new_feature_column_name] = user_base_df['total_page_turn_count'] / user_base_df['lifetime_brochur_view_count']
user_base_df[new_feature_column_name].fillna(0,inplace=True)
new_feature_column_name = ''



---


**NO MORE FEATURES**


---

In [ ]:
#  final dataset here
user_base_df.info()

# **Labeling Data**

In [ ]:
# merging data
# calculating total app start count for each user
new_feature_column_name = 'churn_label' # to keep it centralized
user_base_df_temp = user_base_df.copy()
user_base_df_temp.drop(new_feature_column_name, axis=1, inplace=True, errors='ignore')

# getting all active users from july
active_users_july = pd.concat([df_app_starts_july['userId'], df_brochure_views_july['userId']]).drop_duplicates().reset_index(drop=True)
active_users_july = pd.DataFrame(active_users_july, columns=['userId'])
active_users_july[new_feature_column_name] = 'not_churn'


# adding churn label to dataset
user_base_df_temp = pd.merge(user_base_df_temp,active_users_july,on='userId',how='left')
user_base_df_temp[new_feature_column_name].fillna('churn', inplace=True) # fill 0 to user's who dont exist in app_start  (means they installed but never opened).
user_base_df[new_feature_column_name] = user_base_df_temp[new_feature_column_name]
new_feature_column_name = ''

In [ ]:
user_base_df.to_csv('ready_dataset.csv')

# **playground**